In [ ]:
import torch
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from torch.utils.data import TensorDataset,DataLoader
from sklearn.metrics import f1_score
from sage.utils import evaluate_sigmoid
from sklearn.metrics import ConfusionMatrixDisplay,classification_report

In [ ]:
from sage.models import ResNetv2,ResBlockv2
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

hyperparameters = {
    'block':ResBlockv2,
    'widthi':[4],
    'depthi':[1],
    'n_output_neurons':1,
    'norm':'layer'
}

model = ResNetv2(**hyperparameters)
optimizer = torch.optim.AdamW(model.parameters(),lr=3e-4)
criterion = torch.nn.BCEWithLogitsLoss()

config = torch.load(f'../results/alsaad_iftakhar_base.pt',map_location='cpu',weights_only=False)

model.load_state_dict(config['model'])
criterion.load_state_dict(config['criterion'])
optimizer.load_state_dict(config['optimizer'])

trainlossi = config['trainlossi']
devlossi = config['devlossi']
testlossi = config['testlossi']
trainf1i = config['trainf1i']
devf1i = config['devf1i']
testf1i = config['testf1i']

# If you already moved the model, you shouldn't need to move the optimizer explicitly,
# but you can check and ensure that it's on the correct device.
for state in optimizer.state.values():
    for k, v in state.items():
        if isinstance(v, torch.Tensor):
            state[k] = v.to(device)

In [ ]:
train_participants = ['alsaad','iftakhar']
test_participant = f'tonmoy'
train_files = [f'../data/alpha_pt_windowsize100/{train_participant}_{phase}.pt' for train_participant in train_participants for phase in ['phase1','phase2']]
train_files = train_files + [f'../data/alpha_pt_windowsize100/{test_participant}_phase1.pt'] * 1
tts = []
for train_file in train_files:
    print(train_file)
    a,b,c,d = train_test_split(*torch.load(train_file,weights_only=True),random_state=0)
    tts.append((a[::1],b[::1],c[::1],d[::1]))
X_train = torch.vstack([tt[0] for tt in tts])
X_dev = torch.vstack([tt[1] for tt in tts])
y_train = torch.vstack([tt[2] for tt in tts])
y_dev = torch.vstack([tt[3] for tt in tts])
print(X_train.shape,X_dev.shape,y_train.shape,y_dev.shape)
trainloader = DataLoader(TensorDataset(X_train,y_train),batch_size=128)
devloader = DataLoader(TensorDataset(X_dev,y_dev),batch_size=128)
testloader = DataLoader(TensorDataset(*torch.load(f'../data/alpha_pt_windowsize100/{test_participant}_phase2.pt',weights_only=True)),batch_size=128)

In [ ]:
model = model.to(device)

for epoch in range(1000):
    running_loss = 0.0
    train_preds, train_labels = [], []

    for i, (Xi, yi) in enumerate(trainloader):
        Xi, yi = Xi.to(device), yi.to(device)
        optimizer.zero_grad()
        logits = model(Xi)
        loss = criterion(logits, yi)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        train_preds.extend(logits.sigmoid().round().detach().cpu().numpy())
        train_labels.extend(yi.cpu().numpy())

    trainlossi.append(running_loss / len(trainloader))
    train_f1 = f1_score(train_labels, train_preds, average='macro')
    trainf1i.append(train_f1)

    with torch.no_grad():
        loss,y_true,y_pred = evaluate_sigmoid(devloader,model,criterion)
        devlossi.append(loss)
        dev_f1 = f1_score(y_true, y_pred, average='macro')
        devf1i.append(dev_f1)
    with torch.no_grad():
        loss,y_true,y_pred = evaluate_sigmoid(testloader,model,criterion)
        testlossi.append(loss)
        test_f1 = f1_score(y_true, y_pred, average='macro')
        testf1i.append(test_f1)

    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))
    ax1.plot(trainlossi)
    ax1.plot(devlossi)
    ax1.plot(testlossi)
    ax2.plot(trainf1i)
    ax2.plot(devf1i)
    ax2.plot(testf1i)
    plt.savefig('loss.jpg')
    plt.close()

In [ ]:
torch.save(model.state_dict(),f'../results/alsaad_iftakhar_base_tonmoy_custom.pt')

In [ ]:
loss,y_true,y_pred = evaluate_sigmoid(trainloader,model,criterion)
ConfusionMatrixDisplay.from_predictions(y_true,y_pred)
print(classification_report(y_true,y_pred))
loss,y_true,y_pred = evaluate_sigmoid(devloader,model,criterion)
ConfusionMatrixDisplay.from_predictions(y_true,y_pred)
print(classification_report(y_true,y_pred))
loss,y_true,y_pred = evaluate_sigmoid(testloader,model,criterion)
ConfusionMatrixDisplay.from_predictions(y_true,y_pred)
print(classification_report(y_true,y_pred))

In [ ]:
perfs = []
for p in ['alsaad','anam','asfiqur','ejaz','iftakhar','will','ashlin','tj','tonmoy']:
    print(p)
    for phase in [1,2]:
        testloader = DataLoader(TensorDataset(*torch.load(f'../data/alpha_pt_windowsize100/{p}_phase{phase}.pt',weights_only=True)),batch_size=128)
        loss,y_true,y_pred = evaluate_sigmoid(testloader,model,criterion)
        perfs.append([f1_score(y_true,y_pred,average='macro'),p,phase])

In [ ]:
import pandas as pd
import seaborn as sns
df = pd.DataFrame(perfs)

In [ ]:
df.describe()

In [ ]:
sns.barplot(data=df,x=1,y=0,hue=2)